In [67]:
import pandas as pd
from pandasql import sqldf




In [46]:
# Create Users table
Users = pd.DataFrame( {
    'user_id': [1, 2, 3, 4],
    'join_date': ['2018-01-01', '2018-02-09', '2018-01-19', '2018-05-21'],
    'favorite_brand': ['Lenovo', 'Samsung', 'LG', 'HP']
})



# Create Orders table
Orders = pd.DataFrame( {
    'order_id': [1, 2, 3, 4, 5, 6],
    'order_date': ['2019-08-01', '2018-08-02', '2019-08-03', '2018-08-04', '2018-08-04', '2019-08-05'],
    'item_id': [4, 2, 3, 1, 1, 2],
    'buyer_id': [1, 1, 2, 4, 3, 2],
    'seller_id': [2, 3, 3, 2, 4, 4]
})



In [48]:
Orders

,order_id,order_date,item_id,buyer_id,seller_id
0,1,2019-08-01,4,1,2
1,2,2018-08-02,2,1,3
2,3,2019-08-03,3,2,3
3,4,2018-08-04,1,4,2
4,5,2018-08-04,1,3,4
5,6,2019-08-05,2,2,4


In [49]:
Users

,user_id,join_date,favorite_brand
0,1,2018-01-01,Lenovo
1,2,2018-02-09,Samsung
2,3,2018-01-19,LG
3,4,2018-05-21,HP


In [51]:
query = " SELECT * FROM Users"
pd.read_sql(query, con)

TypeError: read_sql() missing 1 required positional argument: 'con'

In [54]:
sqldf(''' 
    

SELECT 
  u.user_id AS buyer_id, 
  u.join_date, 
  COUNT(o.order_id) AS orders_in_2019 
FROM 
  Users u 
  LEFT JOIN Orders o 
    ON u.user_id = o.buyer_id AND o.order_date LIKE '2019%' 
GROUP BY 
  u.user_id 

 ''')

# Notice that 
# •	In this query, the filtering condition (o.order_date LIKE '2019%') is part of the ON clause 
# of the LEFT JOIN.
#	•	This means that the query will try to match Users with Orders where order_date starts with 
# 2019. If there’s a match, it will return those rows, but if there is no order for a user or
#  no order from 2019, the user will still be included in the result with a NULL value for
#  o.order_id (due to the LEFT JOIN).

# If i had, instead, used a WHERE conditions, I'd filter after the join, removing the observations 
# where orders_in_2019 = 0

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


176. Second highest Salary

Table: Employee

+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| salary      | int  |
+-------------+------+
id is the primary key (column with unique values) for this table.
Each row of this table contains information about the salary of an employee.
 

Write a solution to find the second highest distinct salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

The result format is in the following example.

Example 1:

Input: 
Employee table:
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
Output: 
+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+
Example 2:

Input: 
Employee table:
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
+----+--------+
Output: 
+---------------------+
| SecondHighestSalary |
+---------------------+
| null                |
+---------------------+

In [72]:
employee = pd.DataFrame({
    'id': [1,2],
    'salary': [100, 100]
})

employee

,id,salary
0,1,100
1,2,100


In [73]:
sqldf(''' 
    

SELECT 
    max(salary) as SecondHighestSalary
FROM
    employee
where salary <(
    select max(salary) from Employee
)

 ''')


,SecondHighestSalary
0,None


In [64]:
# First, get only distinct salaries

aux = employee.drop_duplicates(['salary'])
aux = aux.sort_values('salary', ascending = False).reset_index()

if aux.shape[0] >= 2:
    aux['SecondHighestSalary'] = aux.loc[1,'salary']
    aux = aux[['SecondHighestSalary']].drop_duplicates()
else:
    aux['SecondHighestSalary'] = None

aux[['SecondHighestSalary']]



,SecondHighestSalary
0,200


In [15]:
aux

,id,salary,SecondHighestSalary
2,3,300,NaN
1,2,200,NaN
0,1,100,NaN
